In [1]:
import requests
import json
import pandas as pd
import numpy as np

In [2]:
url = 'http://apis.data.go.kr/1360000/FmlandWthrInfoService/getDayStatistics'
servicekey_en = 'f8%2FAQMI%2FVCCLb%2FOjO4hPfmZwE9qdtIrRD07dhQgRjR9OjCkyAsjEzJIBm7raIgOlA4GCw%2FGJp7bytgeT2lYcrw%3D%3D'
servicekey_de = 'f8/AQMI/VCCLb/OjO4hPfmZwE9qdtIrRD07dhQgRjR9OjCkyAsjEzJIBm7raIgOlA4GCw/GJp7bytgeT2lYcrw=='

In [3]:
numOfrows = '3000' #페이지당 결과물 출력 수
start_date = '20100101' #조회 시작일 (2010년 1월 1일)
end_date = '20210930' #조회 마지막일(2021년 10월 1일)
area_id = '999999999' #조회 지역 코드 , 999999999 입력시 전체지역
product_id = 'PA999999' #주산지 작물별 특성 아이디(PA999999 입력시 전체 작물)

In [4]:
product_key = {'배추' : 'PA170401', '무' : 'PA130201', '양파' : 'PA250201', '건고추' : 'PA030101', '마늘' : 'PA120101', '대파' : 'PA090101',
              '얼갈이배추' : 'PA260101', '양배추' : 'PA240101', '깻잎' : 'PA050101', '시금치' : 'PA220101', '미나리' : 'PA140101',
              '당근': 'PA080101', '캠벨얼리' : 'PA340101'}
product_key_test = {'배추' : 'PA170401'}

In [5]:
# *대상품목(16): 배추, 무, 양파, 건고추, 마늘, 대파, 얼갈이배추, 양배추, 깻잎, 시금치, 미나리, 당근, 파프리카, 새송이, 팽이버섯, 토마토
# *대상품종(5): 청상추, 백다다기, 애호박, 캠벨얼리, 샤인마스캇

In [6]:
# product_key = {배추 : 'PA170401', 무 : 'PA130201', 양파 : 'PA250201', 건고추 : 'PA030101', 마늘 : 'PA120101', 대파 : 'PA090101',
#               얼갈이배추 : 'PA260101', 양배추 : 'PA240101', 깻잎 : 'PA050101', 시금치 : 'PA220101', 미나리 : 'PA140101',
#               당근: 'PA080101', 캠벨얼리 : 'PA340101'} #파프리카, 새송이 팽이버섯 토마토, 청상추, 백다다기, 애호박, 샤인마스캇은  전처리 후 , 지역이름을 따야할듯


In [7]:
# #영천 포도-캠벨얼리
# params ={'serviceKey' : servicekey_de, 'pageNo' : '1', 'numOfRows' : numOfrows , 'dataType' : 'JSON', 'ST_YMD' : start_date, 'ED_YMD' : end_date, 'AREA_ID' : area_id, 'PA_CROP_SPE_ID' : product_id }

In [8]:
product_data = None # 데이터 받기의 최종 결과물이 될 녀석
flag = True 

In [ ]:
#상품 종목별 데이터 갯수
for key, value in product_key.items():
    params ={'serviceKey' : servicekey_de, 'pageNo' : '1', 'numOfRows' : numOfrows , 'dataType' : 'JSON', 'ST_YMD' : start_date, 'ED_YMD' : end_date, 'AREA_ID' : area_id, 'PA_CROP_SPE_ID' : value }
    response = requests.get(url, params=params)
    data_api = json.loads(response.text) #딕셔너리 객체로 변환
    count_val = data_api['response']['body']['totalCount']
    print(key, count_val)

In [ ]:
#종목별 데이터 합치기
for key, value in product_key_test.items(): #종목별 데이터 페이지 숫자를 알아내기 위한 for 문
    params ={'serviceKey' : servicekey_de, 'pageNo' : '1', 'numOfRows' : numOfrows , 'dataType' : 'JSON', 'ST_YMD' : start_date, 'ED_YMD' : end_date, 'AREA_ID' : area_id, 'PA_CROP_SPE_ID' : value }
    response = requests.get(url, params=params)
    data_api = json.loads(response.text)
    count_val = data_api['response']['body']['totalCount']
    print(key, count_val)
    numOfpage = int(count_val)//int(numOfrows)
    for i in range(1,numOfpage+2): #종목별 데이터를 페이지별로 모두 추출 및 합병
        print(key, i)
        params ={'serviceKey' : servicekey_de, 'pageNo' : str(i), 'numOfRows' : numOfrows , 'dataType' : 'JSON', 'ST_YMD' : start_date, 'ED_YMD' : end_date, 'AREA_ID' : area_id, 'PA_CROP_SPE_ID' : value }
        response = requests.get(url, params=params)
        data_api_add = json.loads(response.text) #딕셔너리 타입으로 나옴
        dict_data_add = data_api_add['response']['body']['items']['item']
        product_data_add = pd.DataFrame(dict_data_add)
        if flag :
            product_data = product_data_add
            flag = False
        else:
            temp = pd.concat([product_data_add,product_data], axis = 0)
            product_data = temp

In [ ]:
product_data #최종 결과물

In [ ]:
#시계열로 나누어진 엑셀 파일을 열때마다 동시계열의 api 호출을 해서 데이터 크기를 줄여야함. 너무 크면 호출 실패